# Extracted Data Exploration and Cleaning


## Introduction

This notebook focuses on exploring and cleaning the extracted dataset of embedded systems products. The dataset contains various attributes of these products, including operating systems, wireless capabilities, certifications, and other technical specifications. Our goal is to standardize and clean this data to make it more suitable for analysis and insights generation.


## Objectives

1. Explore the extracted data to understand its structure and content
2. Understand the initial extraction data, in each column
3. Clean and standardize the columns
4. Provide summary statistics for each cleaned column
5. Generate example values of the cleaned data, and save the cleaned data to a new csv file


## 1. Setup and Data Loading <a id="1"></a>

We start by importing necessary libraries and loading the result data files.


In [18]:
import os
import re
import ast
import logging
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.max_rows', 1000)

# Set up display options and styles
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

# Configure logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

In [19]:
# Load the data
data_dir = "../data/processed_feature_extraction_results.csv"
df = pd.read_csv(data_dir)
df.head()

,product_id,name,manufacturer,form_factor,evaluation_or_commercialization,processor_architecture,processor_core_count,processor_manufacturer,processor_tdp,memory,onboard_storage,input_voltage,io_count,wireless,operating_system_bsp,operating_temperature_max,operating_temperature_min,certifications,short_summary,full_summary,full_product_description,target_applications,duplicate_ids
0,iW-G27S-SCQM-4L004G-E016G-BIC,IMX QUAD MAX QUAD PLUS PICO ITX SBC,IWAVE SYSTEMS,PICO ITX,False,ARM,6,NXP,NaN,Up to 8GB LPDDR4,Up to 64GB eMMC,12V,"['USB 3.0', 'USB 2.0', 'HDMI', 'Ethernet', 'PCIe', 'CAN', 'UART', 'SPI', 'I2C', 'MIPI CSI', 'MIPI DSI', 'LVDS']","['Wi-Fi', 'Bluetooth']","['Linux', 'Android Pie', 'QNX']",85°C,-40°C,"['RoHS', 'REACH']","The iMX Quad Max Quad Plus Pico ITX SBC is a high-performance, compact single board computer designed for industrial and embedded applications.","The iMX Quad Max Quad Plus Pico ITX SBC integrates dual Cortex-A72 and Cortex-A53 cores, dual GPUs, and a VPU for enhanced multimedia capabilities. It supports up to 8GB LPDDR4 memory and 64GB eMMC storage, with extensive I/O options including USB, HDMI, Ethernet, and wireless connectivity.","The iMX Quad Max Quad Plus Pico ITX SBC is engineered for high-performance applications in industrial, automotive, and medical domains. It features a robust set of interfaces including dual Ethernet, multiple USB ports, PCIe, and advanced display outputs like HDMI and MIPI DSI. The board supports Linux, Android Pie, and QNX operating systems, making it versatile for various development needs.","['Remote Energy Management', 'Intelligent Edge', 'Augmented and Virtual Reality', '4K Media Streaming', 'Industrial Automation', 'Automotive eCockpit']","['iW-G27S-SCQM-4L004G-E016G-BIC', 'iW-G27S-SCQM-4L008G-E032G-BIC']"
1,conga-TC170slash3955U,CONGATC,CONGATEC,COM EXPRESS COMPACT,True,X86,2,INTEL,15W,Up to 32 GByte dual channel DDR4,Optional eMMC 5.1 on board mass storage,21 VDC,"['x PCI Express GEN 3 lanes', 'x Serial ATA Gen 3', 'x USB 3.0', 'x USB 2.0', 'LPC bus', 'I2C bus', 'x UART', 'Digital High Definition Audio Interface']",Available,"['Microsoft Windows', 'Microsoft Windows IoT Enterprise', 'Linux', 'Microsoft Windows Embedded Standard']",60°C,0°C,"FCC Class A, UL 94V0","Compact module with Intel Core processors, dual channel DDR4 memory, and extensive I/O options.","The congaTC is a COM Express Compact module featuring Intel Core i7/i5/i3 and Celeron processors, supporting dual channel DDR4 memory, and offering a wide range of I/O interfaces including PCIe, SATA, USB, and more.","The congaTC module is designed for high-performance applications, equipped with Intel's 6th generation Core processors and Celeron options. It supports up to 32 GByte of dual channel DDR4 memory, and provides a variety of interfaces such as PCIe, SATA, USB, and audio. The module is suitable for industrial and embedded applications, offering features like Intel Turbo Boost, Hyper-Threading, and virtualization technologies.","['Industrial automation', 'Embedded systems', 'Digital signage', 'Medical devices', 'Gaming']",['conga-TC170slash3955U']
2,34012-0416-N1-2,COMEMEL E,KONTRON,COM EXPRESS MINI,True,X86,Up to 14 cores,INTEL,31W,UP TO 8 GBYTE LPDDR,UP TO 64 GBYTE EMMC,5V,"['Up to 2x USB 3.0, 8x USB 2.0, up to 4x serial interfaces or CAN bus']",Supports wireless technologies with miniature RF connectors for WLAN and Bluetooth,"['Windows', 'Linux', 'VxWorks']",85°C,-40°C,EN50155:2021 for Edge AI Platforms,"COM Express mini Type 10 module with Intel Atom, Pentium, and Celeron processors.","The COMemEL E is a COM Express mini Type 10 module featuring Intel Atom, Pentium, and Celeron processors, designed for low power and high performance in industrial applications.","The COMemEL E module is a compact, low-power solution optimized for industrial applications. It supports up to 8 GByte LPDDR memory with In-Band ECC, multiple USB ports, SATA, eMMC Flash, and optional GbE with TSN support. It is designed 

In [20]:
df.columns

Index(['product_id', 'name', 'manufacturer', 'form_factor', 'evaluation_or_commercialization', 'processor_architecture', 'processor_core_count', 'processor_manufacturer', 'processor_tdp', 'memory', 'onboard_storage', 'input_voltage', 'io_count', 'wireless', 'operating_system_bsp', 'operating_temperature_max', 'operating_temperature_min', 'certifications', 'short_summary', 'full_summary', 'full_product_description', 'target_applications', 'duplicate_ids'], dtype='object')

In [21]:
columns = [
    "name",
    "manufacturer",
    "form_factor",
    "evaluation_or_commercialization",
    "processor_architecture",
    "processor_core_count",
    "processor_manufacturer",
    "processor_tdp",
    "memory",
    "onboard_storage",
    "input_voltage",
    "io_count",
    "wireless",
    "operating_system_bsp",
    "operating_temperature_max",
    "operating_temperature_min",
    "certifications",
]


# Display top 10 unique values for each column, sorted by frequency
for column in columns:
    unique_count = df[column].nunique()
    value_counts = df[column].value_counts().head(50)

    print(f"\n{'-' * 100}")
    print(f"{column.upper()} (Total unique values: {unique_count})")
    print(f"{'-' * 100}")

    if not value_counts.empty:
        max_value_length = max(len(str(value)) for value in value_counts.index)
        max_count_length = max(len(str(count)) for count in value_counts.values)

        for value, count in value_counts.items():
            percentage = (count / len(df)) * 100
            print(f"{str(value):<{max_value_length}} | {count:>{max_count_length}} | {percentage:.2f}%")
    else:
        print("No data available for this column")

    print(f"{'-' * 100}\n")


----------------------------------------------------------------------------------------------------
NAME (Total unique values: 677)
----------------------------------------------------------------------------------------------------
AIMB                                                    | 47 | 4.80%
SOM                                                     | 19 | 1.94%
ASMB                                                    | 12 | 1.23%
PCM                                                     | 11 | 1.12%
ARK                                                     | 10 | 1.02%
MIO                                                     | 10 | 1.02%
RSB                                                     |  9 | 0.92%
RASPBERRY PI MODEL B                                    |  7 | 0.72%
AIMB-275                                                |  7 | 0.72%
VENICE GW                                               |  7 | 0.72%
COM EXPRESS COMPACT MODULE                              |  6 | 0.61%
COM EX

## 2. Data Cleaning

### 2.1 Name


This cleaning process will:
- Convert all names to uppercase.
- Remove extra spaces.
- Remove common suffixes that don't add much distinguishing information.
- Standardize common terms and abbreviations.
- Remove parentheses and their contents.
- Remove any remaining non-alphanumeric characters except hyphens and spaces.
- Remove any extra spaces that might have been introduced during the cleaning process.

In [22]:
def clean_product_name(name):
    # Convert to uppercase
    name = name.upper()

    # Remove extra spaces
    name = " ".join(name.split())

    # Remove common suffixes
    suffixes_to_remove = [
        " MODULE",
        " BOARD",
        " COMPUTER",
        " SERIES",
        " MOTHERBOARD",
        " CPU",
        " SINGLE",
        " DEVELOPMENT KIT",
        " DEVELOPMENT",
        " PLATFORM",
        " SYSTEM",
    ]
    for suffix in suffixes_to_remove:
        if name.endswith(suffix):
            name = name[: -len(suffix)]

    # Standardize common prefixes and mid-string occurrences
    replacements = {
        "COM EXPRESS": "COM-EXPRESS",
        "RASPBERRY PI": "RASPBERRY-PI",
        "COMPUTER ON": "COM",
        "TX COMPUTER ON": "TX COM",
        "ICES COM EXPRESS": "ICES COM-EXPRESS",
        "SYSTEM ON": "SOM",
        "COMPUTER-ON-MODULE": "COM",
        "MINI ITX": "MINI-ITX",
        "PICO ITX": "PICO-ITX",
        "MICRO ITX": "MICRO-ITX",
        "NANO ITX": "NANO-ITX",
        "QSEVEN": "Q7",
        "IMXM": "IMX-M",
        "IMX M": "IMX-M",
        "IMXUL": "IMX-UL",
        "IMX UL": "IMX-UL",
    }
    for old, new in replacements.items():
        name = name.replace(old, new)

    # Remove parentheses and their contents
    name = re.sub(r"\([^)]*\)", "", name)

    # Remove any remaining non-alphanumeric characters except hyphen and space
    name = re.sub(r"[^A-Z0-9\- ]", "", name)

    # Remove extra spaces again (in case any were introduced)
    name = " ".join(name.split())

    return name


# Apply the cleaning function
original_unique = df["name"].nunique()
df["name"] = df["name"].apply(clean_product_name)
cleaned_unique = df["name"].nunique()

# Compare the number of unique values before and after cleaning
print(f"\nOriginal unique names: {original_unique}")
print(f"Cleaned unique names: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["name"].value_counts().head(20)


Original unique names: 677
Cleaned unique names: 671
Reduction: 6 (0.89%)


name
AIMB                    47
SOM                     20
ASMB                    12
PCM                     12
ARK                     11
MIO                     10
RSB                      9
RASPBERRY-PI MODEL B     7
VENICE GW                7
AIMB-275                 7
EDHMIC                   6
COM-EXPRESS COMPACT      6
COM-EXPRESS BASIC        6
TREK                     6
MIC                      6
EMETXEI                  5
PCA                      5
UNOG                     5
AIMB-580                 5
CONGATC                  5
Name: count, dtype: int64


### 2.2 Manufacturer


This cleaning process will:
- Convert all manufacturer names to uppercase.
- Remove extra spaces.
- Standardize common variations of the same manufacturer (e.g., combining "ADLINK TECHNOLOGY INC" and "ADLINK" into "ADLINK TECHNOLOGY").

In [23]:
def clean_manufacturer(value):
    if isinstance(value, str):
        # Convert to uppercase
        value = value.upper()

        # Remove extra spaces
        value = " ".join(value.split())

        # Standardize common names
        replacements = {
            "ADLINK TECHNOLOGY INC": "ADLINK TECHNOLOGY",
            "ADLINK": "ADLINK TECHNOLOGY",
            "IWAVE SYSTEMS TECHNOLOGIES": "IWAVE SYSTEMS",
            "IWAVE SYSTEMS TECHNOLOGIES PVT LTD": "IWAVE SYSTEMS",
            "IWAVE SYSTEMS TECH": "IWAVE SYSTEMS",
            "SOLIDRUN LTD": "SOLIDRUN",
            "IBASE TECHNOLOGY": "IBASE",
            "RASPBERRY PI TRADING LTD": "RASPBERRY PI",
            "RASPBERRY PI LTD": "RASPBERRY PI",
            "RASPBERRY PI FOUNDATION": "RASPBERRY PI",
            "CONGATEC GMBH": "CONGATEC",
            "CONGATEC AG": "CONGATEC",
            "VERSALOGIC CORPORATION": "VERSALOGIC",
            "DIGI INTERNATIONAL INC": "DIGI INTERNATIONAL",
            "KARO ELECTRONICS GMBH": "KARO ELECTRONICS",
            "FORLINX EMBEDDED TECHNOLOGY CO LTD": "FORLINX EMBEDDED TECHNOLOGY",
            "SHANGHAI EDA TECHNOLOGY CO LTD": "EDA TECHNOLOGY",
            "ESPRESSIF SYSTEMS": "ESPRESSIF",
            "SEEED STUDIO": "SEEEDSTUDIO",
            "ADVANTECH CO LTD": "ADVANTECH",
            "ADVANTECH INNOCORE": "ADVANTECH",
            "INTRINSYC TECHNOLOGIES CORPORATION": "INTRINSYC TECHNOLOGIES",
            "INTRINSYC TECHNOLOGIES CORP": "INTRINSYC TECHNOLOGIES",
            "SECO SPA": "SECO",
            "KUNBUS GMBH": "KUNBUS",
            "GATEWORKS CORPORATION": "GATEWORKS",
            "MYIR ELECTRONICS LIMITED": "MYIR ELECTRONICS",
            "ACURA EMBEDDED SYSTEMS INC": "ACURA EMBEDDED SYSTEMS",
            "WINSYSTEMS INC": "WINSYSTEMS",
            "AUVIDEA GMBH": "AUVIDEA",
            "MOXA INC": "MOXA",
        }

        for old, new in replacements.items():
            if value == old:
                return new

        return value
    return value


original_unique = df["manufacturer"].nunique()
df["manufacturer"] = df["manufacturer"].apply(clean_manufacturer)
cleaned_unique = df["manufacturer"].nunique()

print(f"\nOriginal unique manufacturers: {original_unique}")
print(f"Cleaned unique manufacturers: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["manufacturer"].value_counts()


Original unique manufacturers: 112
Cleaned unique manufacturers: 88
Reduction: 24 (21.43%)


manufacturer
ADVANTECH                        406
IEI                               60
CONGATEC                          50
KONTRON                           37
VERSALOGIC                        35
ADLINK TECHNOLOGY                 29
IBASE                             27
ARBOR TECHNOLOGY                  25
IWAVE SYSTEMS                     22
RASPBERRY PI                      21
SOLIDRUN                          19
KARO ELECTRONICS                  17
MYIR ELECTRONICS                  15
AXIOMTEK                          14
GATEWORKS                         12
EUROTECH                          11
PHOENIX CONTACT                   11
EDA TECHNOLOGY CO LTD              9
SECO                               9
NEXCOBOT                           9
DIGI INTERNATIONAL                 8
GIGAIPC                            6
FORLINX EMBEDDED TECHNOLOGY        6
SEEEDSTUDIO                        6
RADXA                              6
ASUS                               6
NVIDIA                   


### 2.3 Form Factor


This cleaning process will:
- Convert all form factors to uppercase.
- Remove extra spaces.
- Standardize common terms and variations.
- Group similar form factors together.
- Remove any remaining non-alphanumeric characters except hyphens and spaces.

In [24]:
def clean_form_factor(value):
    if not isinstance(value, str):
        return value

    # Convert to uppercase
    value = value.upper()

    # Remove extra spaces
    value = " ".join(value.split())

    # Standardize common terms
    replacements = {
        "MINI-ITX": "MINI-ITX",
        "MINI ITX": "MINI-ITX",
        "MINIITX": "MINI-ITX",
        "THIN MINI-ITX": "THIN MINI-ITX",
        "THIN MINIITX": "THIN MINI-ITX",
        "MICRO-ATX": "MICRO-ATX",
        "MICRO ATX": "MICRO-ATX",
        "MICROATX": "MICRO-ATX",
        "MATX": "MICRO-ATX",
        "PICO-ITX": "PICO-ITX",
        "PICO ITX": "PICO-ITX",
        "PICOITX": "PICO-ITX",
        "NANO-ITX": "NANO-ITX",
        "NANO ITX": "NANO-ITX",
        "NANOITX": "NANO-ITX",
        "COM EXPRESS": "COM EXPRESS",
        "COMEXPRESS": "COM EXPRESS",
        "COM-EXPRESS": "COM EXPRESS",
        "QSEVEN": "QSEVEN",
        "Q7": "QSEVEN",
        "SMARC": "SMARC",
        "SINGLE BOARD COMPUTER": "SBC",
        "SINGLEBOARD COMPUTER": "SBC",
        "SINGLE-BOARD COMPUTER": "SBC",
        "SINGLEBOARD": "SBC",
        "EMBEDDED SBC": "SBC",
        "SYSTEM ON MODULE": "SOM",
        "SYSTEM-ON-MODULE": "SOM",
        "SYSTEM MODULE": "SOM",
        "COMPUTER ON MODULE": "COM",
        "RASPBERRY PI": "RASPBERRY PI",
        "RASPBERRY PI COMPATIBLE": "RASPBERRY PI",
        "FANLESS BOX PC": "BOX PC",
        "FANLESS EMBEDDED BOX PC": "BOX PC",
        "EMBEDDED BOX PC": "BOX PC",
        "EMBEDDED BOX COMPUTER": "BOX PC",
        "BOX COMPUTER": "BOX PC",
        "FANLESS PC": "BOX PC",
        "FANLESS COMPACT SYSTEM": "BOX PC",
        "COMPACT EXPANDABLE FANLESS SYSTEM": "BOX PC",
        "DINRAIL": "DIN RAIL",
        "DIN RAIL MOUNTABLE": "DIN RAIL",
        "PICMG": "PICMG",
        "FULLSIZE PICMG": "PICMG FULLSIZE",
        "PICMG FULLSIZE CPU CARD": "PICMG FULLSIZE",
        "HALF-SIZE PICMG": "PICMG HALF-SIZE",
        "HALFSIZE PICMG": "PICMG HALF-SIZE",
        "COMPACTPCI": "COMPACT PCI",
        "MINI PCIE": "MINI PCIE",
        "HALF-SIZE": "HALF-SIZE",
        "HALFSIZE": "HALF-SIZE",
        "HALF SIZE": "HALF-SIZE",
        "ALL-IN-ONE": "ALL-IN-ONE",
        "RACKMOUNT": "RACKMOUNT",
        "U RACKMOUNT": "RACKMOUNT",
        "U RACKMOUNT CHASSIS": "RACKMOUNT",
        "EMBEDDED": "EMBEDDED",
        "EMBEDDED SYSTEM": "EMBEDDED",
        "EMBEDDED IPC": "EMBEDDED",
        "EMBEDDED MODULE": "EMBEDDED",
    }

    for old, new in replacements.items():
        if value == old or value.startswith(old + " "):
            return new

    # Remove any remaining non-alphanumeric characters except hyphen and space
    value = re.sub(r"[^A-Z0-9\- ]", "", value)

    return value


original_unique = df["form_factor"].nunique()
df["form_factor"] = df["form_factor"].apply(clean_form_factor)
cleaned_unique = df["form_factor"].nunique()

print(f"\nOriginal unique form factors: {original_unique}")
print(f"Cleaned unique form factors: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["form_factor"].value_counts()


Original unique form factors: 227
Cleaned unique form factors: 140
Reduction: 87 (38.33%)


form_factor
COM EXPRESS                         147
SBC                                 133
MINI-ITX                             76
ATX                                  51
SMARC                                40
BOX PC                               38
QSEVEN                               37
MICRO-ATX                            35
PICO-ITX                             35
SOM                                  31
RASPBERRY PI                         25
SODIMM                               17
DIN RAIL                             13
EBX                                  12
PCPLUS                               12
EPIC                                 11
EMBEDDED                             11
HALF-SIZE                            11
QFN                                  10
ETX                                   9
PICMG                                 9
COM                                   8
RACKMOUNT                             8
ALL-IN-ONE                            8
THIN MINI-ITX               

### 2.4 Evaluation or Commercialization


In [25]:
def clean_evaluation_or_commercialization(value):
    if isinstance(value, str):
        value = value.lower()
        if value in ["true", "false"]:
            return value == "true"
    return None

df["evaluation_or_commercialization"] = df["evaluation_or_commercialization"].apply(clean_evaluation_or_commercialization)

df["evaluation_or_commercialization"].value_counts()

evaluation_or_commercialization
False    826
True     151
Name: count, dtype: int64

### 2.5 Processor Architecture

This cleaning process will:
- Convert all processor architectures to uppercase.
- Remove extra spaces.
- Standardize common terms and variations.
- Group similar architectures together.
- Handle specific cases like Intel and ARM variations.

In [26]:
def clean_processor_architecture(value):
    if not isinstance(value, str):
        return value

    # Convert to uppercase
    value = value.upper()

    # Remove extra spaces
    value = " ".join(value.split())

    # Standardize common terms
    replacements = {
        "X86": "X86",
        "X86-64": "X86-64",
        "X86_64": "X86-64",
        "INTEL 64": "X86-64",
        "IA-32": "X86",
        "ARM": "ARM",
        "ARM CORTEX-A53": "ARM CORTEX-A53",
        "ARM CORTEX A53": "ARM CORTEX-A53",
        "CORTEX-A53 64-BIT": "ARM CORTEX-A53",
        "ARM CORTEX-A": "ARM CORTEX-A",
        "ARM CORTEX A": "ARM CORTEX-A",
        "CORTEX-A": "ARM CORTEX-A",
        "ARM CORTEXA": "ARM CORTEX-A",
        "ARM CORTEX-A72": "ARM CORTEX-A72",
        "ARM CORTEX-A8": "ARM CORTEX-A8",
        "ARM CORTEX-A9": "ARM CORTEX-A9",
        "ARM CORTEX-A7": "ARM CORTEX-A7",
        "CORTEX-A7": "ARM CORTEX-A7",
        "ARM CORTEX-M3": "ARM CORTEX-M3",
        "ARM CORTEX-M4": "ARM CORTEX-M4",
        "ARM CORTEX-M33": "ARM CORTEX-M33",
        "ARMV8": "ARMV8",
        "ARM BIG.LITTLE": "ARM BIG.LITTLE",
        "ARM CORTEX-A72/A53": "ARM BIG.LITTLE",
        "SILVERMONT": "INTEL ATOM",
        "APOLLO LAKE": "INTEL ATOM",
        "BAY TRAIL": "INTEL ATOM",
        "INTEL ATOM C3000": "INTEL ATOM",
        "ZEN": "AMD ZEN",
        "SKYLAKE-U": "INTEL CORE",
        "HASWELL": "INTEL CORE",
        "ALDER LAKE": "INTEL CORE",
        "ALDER LAKE-S": "INTEL CORE",
        "ALDER LAKE N": "INTEL CORE",
        "RAPTOR LAKE-S": "INTEL CORE",
        "INTEL PERFORMANCE HYBRID ARCHITECTURE": "INTEL HYBRID",
        "PERFORMANCE HYBRID ARCHITECTURE": "INTEL HYBRID",
        "X86 HYBRID ARCHITECTURE": "INTEL HYBRID",
        "INTEL® CORE™ ULTRA METEOR LAKE-H/U": "INTEL HYBRID",
        "RISCV": "RISC-V",
        "XTENSA": "XTENSA",
        "XTENSA LX6": "XTENSA",
        "MOBILE ATHLON ARCHITECTURE": "AMD",
        "8-BIT": "8-BIT",
        "16-BIT": "16-BIT",
        "32-BIT ARM ARCHITECTURE": "ARM",
    }

    for old, new in replacements.items():
        if value == old or value.startswith(old + " "):
            return new

    # If no match found, return the original value
    return value


# Apply the cleaning function
original_unique = df["processor_architecture"].nunique()
df["processor_architecture"] = df["processor_architecture"].apply(clean_processor_architecture)
cleaned_unique = df["processor_architecture"].nunique()

print(f"\nOriginal unique processor architectures: {original_unique}")
print(f"Cleaned unique processor architectures: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["processor_architecture"].value_counts()


Original unique processor architectures: 60
Cleaned unique processor architectures: 19
Reduction: 41 (68.33%)


processor_architecture
X86                                                         626
ARM                                                         291
X86-64                                                       11
INTEL ATOM                                                    9
INTEL CORE                                                    7
INTEL HYBRID                                                  4
AMD ZEN                                                       3
INTEL                                                         3
RISC-V                                                        2
XTENSA                                                        2
ARM CORTEX-A7                                                 1
PICMZ                                                         1
8-BIT                                                         1
ARM CORTEX-A53                                                1
AMD                                                           1
ARMV8            

### 2.6 Processor Core Count

In [10]:
def clean_processor_core_count(value):
    if isinstance(value, str):
        value = value.lower()
        value = re.sub(r"\s+", " ", value)  # Remove extra spaces

        # Handle specific mappings
        mapping = {
            "single": "1", "dual": "2", "quad": "4", "hexa": "6", "octa": "8",
            "four": "4", "eight": "8", "nine": "9"
        }
        for key, replacement in mapping.items():
            if key in value:
                return replacement

        # Handle ranges
        range_match = re.search(r"(\d+)\s*(?:to|-)\s*(\d+)", value)
        if range_match:
            return f"{range_match.group(1)}-{range_match.group(2)}"

        # Handle "up to X cores"
        up_to_match = re.search(r"up to (\d+)", value)
        if up_to_match:
            return f"1-{up_to_match.group(1)}"

        # Handle "X to Y cores"
        to_match = re.search(r"(\w+)\s+to\s+(\w+)\s+cores", value)
        if to_match:
            start = mapping.get(to_match.group(1), to_match.group(1))
            end = mapping.get(to_match.group(2), to_match.group(2))
            return f"{start}-{end}"

        # Handle "X processing cores"
        processing_cores_match = re.search(r"(\w+)\s+processing\s+cores", value)
        if processing_cores_match:
            return mapping.get(processing_cores_match.group(1), processing_cores_match.group(1))

        # Extract single number if present
        number_match = re.search(r"\d+", value)
        if number_match:
            return number_match.group()

    return value


original_unique = df["processor_core_count"].nunique()
df["processor_core_count"] = df["processor_core_count"].apply(clean_processor_core_count)
cleaned_unique = df["processor_core_count"].nunique()

print(f"\nOriginal unique processor core counts: {original_unique}")
print(f"Cleaned unique processor core counts: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["processor_core_count"].value_counts()



Original unique processor core counts: 141
Cleaned unique processor core counts: 30
Reduction: 111 (78.72%)


processor_core_count
4       460
2       202
1        60
6        41
8        37
24       15
16       15
1-14     12
14       10
1-16     10
1-24      9
6-10      6
12        6
1-4       5
10        5
9         2
1-8       2
128       2
1-6       2
1-28      2
80        1
3         1
64        1
5         1
32        1
2-16      1
1-60      1
20        1
1-10      1
1-32      1
Name: count, dtype: int64

### 2.7 Processor Manufacturer

This cleaning process will:
- Convert all manufacturer names to uppercase.
- Remove extra spaces.
- Standardize common terms and variations.
- Group similar manufacturers together.

In [11]:
def clean_processor_manufacturer(value):
    if not isinstance(value, str):
        return value

    # Convert to uppercase
    value = value.upper()

    # Remove extra spaces
    value = " ".join(value.split())

    # Standardize common terms
    replacements = {
        "INTEL": "INTEL",
        "NXP": "NXP",
        "AMD": "AMD",
        "BROADCOM": "BROADCOM",
        "ROCKCHIP": "ROCKCHIP",
        "FREESCALE": "FREESCALE",
        "ARM": "ARM",
        "ARM LIMITED": "ARM",
        "TEXAS INSTRUMENTS": "TEXAS INSTRUMENTS",
        "TI": "TEXAS INSTRUMENTS",
        "NVIDIA": "NVIDIA",
        "ALLWINNER": "ALLWINNER",
        "ALLWINNER TECH": "ALLWINNER",
        "ALLWINNER TECHNOLOGY": "ALLWINNER",
        "XILINX": "XILINX",
        "RENESAS": "RENESAS",
        "DMP": "DMP",
        "DMP ELECTRONICS INC": "DMP",
        "QUALCOMM": "QUALCOMM",
        "QUALCOMM TECHNOLOGIES INC": "QUALCOMM",
        "ESPRESSIF SYSTEMS": "ESPRESSIF",
        "ESPRESSIF": "ESPRESSIF",
        "MEDIATEK": "MEDIATEK",
        "MARVELL": "MARVELL",
        "STMICROELECTRONICS": "STMICROELECTRONICS",
        "ST MICROELECTRONICS": "STMICROELECTRONICS",
        "ST": "STMICROELECTRONICS",
        "STM": "STMICROELECTRONICS",
        "RASPBERRY PI LTD": "RASPBERRY PI",
        "RASPBERRY PI": "RASPBERRY PI",
        "KNERON": "KNERON",
        "STERICSSON": "STERICSSON",
        "RABBIT": "RABBIT",
        "RABBIT SEMICONDUCTOR": "RABBIT",
        "RABBIT SEMICONDUCTOR INC": "RABBIT",
        "WESTERN DESIGN CENTER": "WESTERN DESIGN CENTER",
        "ADVANTECH": "ADVANTECH",
        "ATMEL": "ATMEL",
        "VIA": "VIA",
        "MICROCHIP": "MICROCHIP",
    }

    for old, new in replacements.items():
        if value == old or value.startswith(old + " "):
            return new

    # If no match found, return the original value
    return value


# Apply the cleaning function
original_unique = df["processor_manufacturer"].nunique()
df["processor_manufacturer"] = df["processor_manufacturer"].apply(clean_processor_manufacturer)
cleaned_unique = df["processor_manufacturer"].nunique()

print(f"\nOriginal unique processor manufacturers: {original_unique}")
print(f"Cleaned unique processor manufacturers: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["processor_manufacturer"].value_counts()


Original unique processor manufacturers: 49
Cleaned unique processor manufacturers: 27
Reduction: 22 (44.90%)


processor_manufacturer
INTEL                    627
NXP                      120
AMD                       61
BROADCOM                  34
ROCKCHIP                  23
TEXAS INSTRUMENTS         15
FREESCALE                 13
ARM                       12
ALLWINNER                 10
NVIDIA                     8
DMP                        7
STMICROELECTRONICS         6
RENESAS                    6
QUALCOMM                   5
XILINX                     5
RABBIT                     4
ESPRESSIF                  4
MARVELL                    4
MEDIATEK                   3
RASPBERRY PI               3
VIA                        1
MICROCHIP                  1
ATMEL                      1
ADVANTECH                  1
WESTERN DESIGN CENTER      1
KNERON                     1
STERICSSON                 1
Name: count, dtype: int64

### 2.8 Processor TDP

This cleaning process will:
- Preserves range values (e.g., "6W-12W")
- Standardizes all numeric values to one decimal place
- Converts "up to X" format to a range starting from 0W
- Handles special cases like "ultra low power" and "low power consumption"
- Removes extra spaces and standardizes the "W" suffix

In [12]:
def clean_processor_tdp(value):
    if not isinstance(value, str):
        return value

    value = value.lower().strip()
    value = re.sub(r"\s+", " ", value)  # Remove extra spaces

    # Handle special cases
    if value in ["ultra low power", "low power consumption", "low power design", "low power"]:
        return "LOW POWER"

    # Convert ranges to standard format
    range_match = re.search(r"(\d+(?:\.\d+)?)\s*(?:w|watts)?\s*(?:to|-)\s*(\d+(?:\.\d+)?)\s*(?:w|watts)?", value)
    if range_match:
        return f"{float(range_match.group(1)):.1f}W-{float(range_match.group(2)):.1f}W"

    # Handle "up to X" format
    up_to_match = re.search(r"(?:up to|max) (\d+(?:\.\d+)?)\s*(?:w|watts)", value)
    if up_to_match:
        return f"0.0W-{float(up_to_match.group(1)):.1f}W"

    # Handle single values
    single_match = re.search(r"(\d+(?:\.\d+)?)\s*(?:w|watts)?", value)
    if single_match:
        return f"{float(single_match.group(1)):.1f}W"

    return value


# Apply the cleaning function
df["processor_tdp"] = df["processor_tdp"].apply(clean_processor_tdp)

# Display the updated value counts
print(df["processor_tdp"].value_counts().head(20))

# Count unique TDP configurations
unique_tdp_configs = df["processor_tdp"].nunique()
print(f"\nNumber of unique TDP configurations: {unique_tdp_configs}")

# Calculate statistics for single TDP values
tdp_values = df["processor_tdp"].dropna()
single_tdps = tdp_values[tdp_values.str.match(r"^\d+\.\d+W$")]
single_tdps_numeric = single_tdps.str.rstrip("W").astype(float)

if not single_tdps_numeric.empty:
    print(f"\nMost common TDP: {single_tdps.mode().values[0]}")
    print(f"Median TDP: {single_tdps_numeric.median():.1f}W")
    print(f"Mean TDP: {single_tdps_numeric.mean():.1f}W")
    print(f"Min TDP: {single_tdps_numeric.min():.1f}W")
    print(f"Max TDP: {single_tdps_numeric.max():.1f}W")

# Count products with TDP ranges
tdp_ranges = tdp_values[tdp_values.str.contains("-")]
print(f"\nNumber of products with TDP ranges: {len(tdp_ranges)}")
print(f"Percentage of products with TDP ranges: {len(tdp_ranges) / len(tdp_values) * 100:.2f}%")

# Count low power products
low_power_count = len(tdp_values[tdp_values == "LOW POWER"])
print(f"\nNumber of products with unspecified low power consumption: {low_power_count}")

processor_tdp
6.0W           102
15.0W           94
35.0W           49
45.0W           45
65.0W           38
10.0W           33
95.0W           26
125.0W          22
28.0W           20
12.0W           16
0.0W-45.0W      12
25.0W           10
6.0W-12.0W       9
35.0W-65.0W      8
LOW POWER        7
5.0W             6
205.0W           5
12.0W-25.0W      5
15.0W-45.0W      5
15.0W-25.0W      4
Name: count, dtype: int64

Number of unique TDP configurations: 83

Most common TDP: 6.0W
Median TDP: 15.0W
Mean TDP: 38.7W
Min TDP: 1.0W
Max TDP: 1900.0W

Number of products with TDP ranges: 84
Percentage of products with TDP ranges: 13.68%

Number of products with unspecified low power consumption: 7


### 2.9 Memory

This cleaning process will:
- Preserves "Up to X" format
- Keeps range values
- Maintains information about memory types (DDR, LPDDR, SDRAM, etc.)
- Standardizes memory sizes to GB or MB
- Handles "Max Capacity X" format
- Keeps original values for entries that don't match any specific format


In [14]:
def standardize_memory_size(value, unit):
    value = float(value)
    if unit.upper() == "KB":
        return f"{value / 1024:.2f}MB"
    elif unit.upper() == "MB":
        return f"{value:.0f}MB"
    elif unit.upper() == "GB":
        return f"{value:.0f}GB"
    elif unit.upper() == "TB":
        return f"{value * 1024:.0f}GB"
    return f"{value}{unit}"


def clean_memory(value):
    if not isinstance(value, str):
        return value

    value = value.upper().strip()
    value = re.sub(r"\s+", " ", value)  # Remove extra spaces

    # Handle "Up to X" format
    up_to_match = re.search(r"UP TO (\d+(?:\.\d+)?)\s*(GB|MB|TB|KB)", value, re.IGNORECASE)
    if up_to_match:
        size = standardize_memory_size(up_to_match.group(1), up_to_match.group(2))
        return f"UP TO {size}"

    # Handle range format
    range_match = re.search(
        r"(\d+(?:\.\d+)?)\s*(GB|MB|TB|KB)\s*-\s*(\d+(?:\.\d+)?)\s*(GB|MB|TB|KB)", value, re.IGNORECASE
    )
    if range_match:
        start = standardize_memory_size(range_match.group(1), range_match.group(2))
        end = standardize_memory_size(range_match.group(3), range_match.group(4))
        return f"{start}-{end}"

    # Handle single values with type
    single_match = re.search(r"(\d+(?:\.\d+)?)\s*(GB|MB|TB|KB)\s*(DDR\d?L?|LPDDR\d?X?|SDRAM)?", value, re.IGNORECASE)
    if single_match:
        size = standardize_memory_size(single_match.group(1), single_match.group(2))
        mem_type = single_match.group(3) or ""
        return f"{size} {mem_type}".strip()

    # Handle "Max Capacity X" format
    max_cap_match = re.search(r"MAX(?:IMUM)?\s+CAPACITY\s+(\d+(?:\.\d+)?)\s*(GB|MB|TB|KB)", value, re.IGNORECASE)
    if max_cap_match:
        size = standardize_memory_size(max_cap_match.group(1), max_cap_match.group(2))
        return f"MAX {size}"

    # If no specific format is matched, return the original value
    return value


# Apply the cleaning function
df["memory"] = df["memory"].apply(clean_memory)

# Display the updated value counts
print(df["memory"].value_counts().head(20))

# Count unique memory configurations
unique_memory_configs = df["memory"].nunique()
print(f"\nNumber of unique memory configurations: {unique_memory_configs}")

# Calculate statistics for single memory values
memory_values = df["memory"].dropna()
single_memories = memory_values[memory_values.str.match(r"^\d+(?:GB|MB)")]
single_memories_gb = single_memories.str.extract("(\d+)(\w+)").apply(
    lambda x: float(x[0]) if x[1] == "GB" else float(x[0]) / 1024, axis=1
)

if not single_memories_gb.empty:
    print(f"\nMost common memory size: {single_memories.mode().values[0] if not single_memories.empty else 'N/A'}")
    print(f"Median memory size: {single_memories_gb.median():.2f}GB")
    print(f"Mean memory size: {single_memories_gb.mean():.2f}GB")
    print(f"Min memory size: {single_memories_gb.min():.2f}GB")
    print(f"Max memory size: {single_memories_gb.max():.2f}GB")

# Count products with memory ranges or "Up to" specifications
memory_ranges = memory_values[memory_values.str.contains("-|UP TO|MAX")]
print(f"\nNumber of products with memory ranges or max specifications: {len(memory_ranges)}")
print(
    f"Percentage of products with memory ranges or max specifications: {len(memory_ranges) / len(memory_values) * 100:.2f}%"
)

# Count products with specific memory types
ddr_count = len(memory_values[memory_values.str.contains("DDR")])
lpddr_count = len(memory_values[memory_values.str.contains("LPDDR")])
sdram_count = len(memory_values[memory_values.str.contains("SDRAM")])
print(f"\nNumber of products with DDR memory: {ddr_count}")
print(f"Number of products with LPDDR memory: {lpddr_count}")
print(f"Number of products with SDRAM: {sdram_count}")

memory
UP TO 32GB     85
UP TO 8GB      78
UP TO 64GB     66
8GB DDR3L      47
8GB            46
4GB LPDDR4     36
UP TO 16GB     34
4GB            31
UP TO 4GB      30
32GB           28
4GB DDR3L      28
1GB            26
64GB           22
1GB DDR3       21
2GB            20
8GB DDR4       20
UP TO 128GB    18
UP TO 2GB      18
2GB LPDDR4     17
512MB DDR3     15
Name: count, dtype: int64

Number of unique memory configurations: 131

Most common memory size: 8GB DDR3L
Median memory size: 4.00GB
Mean memory size: 15.90GB
Min memory size: 0.00GB
Max memory size: 768.00GB

Number of products with memory ranges or max specifications: 359
Percentage of products with memory ranges or max specifications: 36.75%

Number of products with DDR memory: 391
Number of products with LPDDR memory: 102
Number of products with SDRAM: 2


### 2.10 Onboard Storage

This enhanced cleaning function will:
- Standardize storage sizes (GB, MB, TB)
- Standardize mentions of eMMC, SSD, and common interfaces (SATA, MSATA, NVME, PCIE, M.2)
- Remove common phrases like "UP TO", "OPTIONAL", "ONBOARD", "SUPPORTS"
- Simplify ranges and multiple options
- Standardize flash types (NAND, NOR, SPI)
- Remove parentheses and their contents
- Remove any remaining non-alphanumeric characters except hyphen, period, and forward slash
- Remove extra spaces

In [14]:
def clean_onboard_storage(value):
    if not isinstance(value, str):
        return value

    value = value.upper()
    value = re.sub(r"\s+", " ", value)  # Remove extra spaces

    # Standardize storage sizes
    value = re.sub(r"(\d+)\s*(GB|MB|TB)", r"\1\2", value)
    value = re.sub(r"(\d+)GBYTE", r"\1GB", value)
    value = re.sub(r"(\d+)MBYTE", r"\1MB", value)
    value = re.sub(r"(\d+)TBYTE", r"\1TB", value)

    # Standardize eMMC mentions
    value = re.sub(r"EMMC", "EMMC", value)

    # Standardize SSD mentions
    value = re.sub(r"(NVME|SATA|M\.2)\s+SSD", r"\1 SSD", value)

    # Standardize common interfaces
    value = re.sub(r"SATA(?: III?)?", "SATA", value)
    value = re.sub(r"MSATA", "MSATA", value)
    value = re.sub(r"NVME", "NVME", value)
    value = re.sub(r"PCIE", "PCIE", value)
    value = re.sub(r"M\.2", "M.2", value)

    # Remove common phrases
    value = re.sub(r"UP TO ", "", value)
    value = re.sub(r"OPTIONAL ", "", value)
    value = re.sub(r"ONBOARD ", "", value)
    value = re.sub(r"SUPPORTS? ", "", value)

    # Simplify ranges
    value = re.sub(r"(\d+)GB?-(\d+)GB", r"\1GB-\2GB", value)

    # Simplify multiple options
    value = re.sub(r"(\d+)GB,?\s+(\d+)GB,?\s+(\d+)GB", r"\1GB/\2GB/\3GB", value)

    # Standardize flash types
    value = re.sub(r"NAND FLASH", "NAND", value)
    value = re.sub(r"NOR FLASH", "NOR", value)
    value = re.sub(r"SPI FLASH", "SPI", value)

    # Remove parentheses and their contents
    value = re.sub(r"\([^)]*\)", "", value)

    # Remove any remaining non-alphanumeric characters except hyphen, period, and forward slash
    value = re.sub(r"[^A-Z0-9\-\./]", " ", value)

    # Remove extra spaces again (in case any were introduced)
    value = " ".join(value.split())

    return value


original_unique = df["onboard_storage"].nunique()
df["onboard_storage"] = df["onboard_storage"].apply(clean_onboard_storage)
cleaned_unique = df["onboard_storage"].nunique()

print(f"\nOriginal unique onboard storage: {original_unique}")
print(f"Cleaned unique onboard storage: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["onboard_storage"].value_counts()


Original unique onboard storage: 508
Cleaned unique onboard storage: 422
Reduction: 86 (16.93%)


onboard_storage
64GB EMMC                                                                82
16GB EMMC                                                                54
MSATA                                                                    49
8GB EMMC                                                                 26
EMMC                                                                     20
32GB EMMC                                                                19
NVME SSD                                                                 19
EMMC 64GB                                                                15
8GB EMMC NAND                                                            11
COMPACTFLASH TYPE III                                                    10
NON-VOLATILE USER DATA STORAGE                                            9
1TB NVME SSD                                                              8
COMPACTFLASH SOCKET                                                     

### 2.11 Input Voltage

This cleaning process will:
- Handles special cases like 'ATX', 'AC', 'VDC', etc.
- Standardizes all ATX-related entries to simply 'ATX'.
- Categorizes entries like "WIDE VOLTAGE INPUT RANGE" as 'VARIABLE'.
- Converts all numeric values to floats for consistency.
- Handles ranges that don't include 'V' at the end.

In [15]:
def clean_input_voltage(value):
    if not isinstance(value, str):
        return value

    value = value.upper().strip()
    value = re.sub(r"\s+", " ", value)  # Remove extra spaces

    # Handle special cases
    if value in ["ATX", "AC", "VDC", "DC POWER", "DC POWER INPUT", "USB POWERED"]:
        return value
    if "ATX" in value or "AT POWER" in value:
        return "ATX"
    if "WIDE VOLTAGE INPUT RANGE" in value or "NOMINAL VOLTAGE" in value:
        return "VARIABLE"

    # Standardize range format
    range_match = re.search(r"(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)\s*V", value)
    if range_match:
        return f"{float(range_match.group(1))}V-{float(range_match.group(2))}V"

    # Handle single values
    single_match = re.search(r"(\d+(?:\.\d+)?)\s*V(?:OLT)?S?", value)
    if single_match:
        return f"{float(single_match.group(1))}V"

    # Handle ranges without 'V'
    range_match_no_v = re.search(r"(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)", value)
    if range_match_no_v:
        return f"{float(range_match_no_v.group(1))}V-{float(range_match_no_v.group(2))}V"

    return value


original_unique = df["input_voltage"].nunique()
df["input_voltage"] = df["input_voltage"].apply(clean_input_voltage)
cleaned_unique = df["input_voltage"].nunique()

print("\n=== Input Voltage Cleaning Results ===")
print(f"Original unique values: {original_unique}")
print(f"Cleaned unique values:  {cleaned_unique}")
print(f"Reduction:              {original_unique - cleaned_unique}")
print(f"Reduction percentage:   {((original_unique - cleaned_unique) / original_unique) * 100:.2f}%")

print("\n=== Top 20 Input Voltage Configurations ===")
print(df["input_voltage"].value_counts().head(20).to_string())

# Count unique voltage configurations
unique_voltage_configs = df["input_voltage"].nunique()
print(f"\nTotal unique voltage configurations: {unique_voltage_configs}")

# Calculate statistics
voltage_values = df["input_voltage"].dropna()
single_voltages = voltage_values[voltage_values.str.contains("^[\d.]+V$", regex=True)].str.rstrip("V").astype(float)

print("\n=== Single Voltage Statistics ===")
print(f"Most common:  {single_voltages.mode().values[0]}V")
print(f"Median:       {single_voltages.median():.2f}V")
print(f"Mean:         {single_voltages.mean():.2f}V")
print(f"Min:          {single_voltages.min():.2f}V")
print(f"Max:          {single_voltages.max():.2f}V")

# Count products with voltage ranges
voltage_ranges = voltage_values[voltage_values.str.contains("-")]
print("\n=== Voltage Range Statistics ===")
print(f"Products with ranges: {len(voltage_ranges)}")
print(f"Percentage:           {len(voltage_ranges) / len(voltage_values) * 100:.2f}%")

print("\n=== Full Input Voltage Distribution ===")
print(df["input_voltage"].value_counts().to_string())


=== Input Voltage Cleaning Results ===
Original unique values: 137
Cleaned unique values:  42
Reduction:              95
Reduction percentage:   69.34%

=== Top 20 Input Voltage Configurations ===
input_voltage
12.0V            513
5.0V             239
3.3V              33
9.0V-36.0V        28
24.0V             23
ATX               21
100.0V-240.0V     17
12.0V-24.0V       14
8.0V-60.0V        12
19.0V              6
8.0V-30.0V         5
3.0V               3
VARIABLE           2
AC                 2
8.5V               2
12.0V-28.0V        2
36.0V              2
12.0V-60.0V        1
8.0V               1
30.0V              1

Total unique voltage configurations: 42

=== Single Voltage Statistics ===
Most common:  12.0V
Median:       12.00V
Mean:         10.09V
Min:          1.20V
Max:          48.00V

=== Voltage Range Statistics ===
Products with ranges: 88
Percentage:           9.27%

=== Full Input Voltage Distribution ===
input_voltage
12.0V                    513
5.0V              

### 2.12 IO Count

This cleaning process will:
- USB standardization:
    - USB 2.0 and USB 2 are now treated as the same.
    - USB 3.0, 3.1, and 3.2 are grouped as "USB 3.x" since the differences are often minimal and inconsistently reported.
    - Unspecified USB remains as just "USB".
- SATA and PCIe versioning:
    - We now capture version information for SATA and PCIe when available.
    - This allows us to differentiate between, for example, SATA II and SATA III.
- Other interfaces:
    - We've kept other interfaces (like ETHERNET, SERIAL, DISPLAY) without version information, as it's less commonly specified and less critical for differentiation.
- Handling ambiguity:
    - When a product just specifies "USB" without a version, we keep it as "USB". This ambiguity is preserved in the data, allowing for further analysis if needed.

In [16]:
def standardize_io_type(io_type):
    io_type = io_type.upper().strip()

    # Standardize USB types
    usb_match = re.search(r"\bUSB\s*([\d.]+)?(?:\s*(?:GEN)?\s*([\d.]+))?", io_type)
    if usb_match:
        version = usb_match.group(1) or usb_match.group(2)
        if version:
            # Standardize version numbers
            if version in ["1", "1.0"]:
                return "USB 1.0"
            if version in ["2", "2.0"]:
                return "USB 2.0"
            elif version in ["3", "3.0"]:
                return "USB 3.0"
            elif version in ["4", "4.0"]:
                return "USB 4.0"
            else:
                return f"USB {version}"
        return "USB"

    # Standardize SATA
    sata_match = re.search(r"\bSATA\s*(I{1,3}|[\d.]+)?", io_type)
    if sata_match:
        version = sata_match.group(1)
        if version:
            if version == "III":
                return "SATA 3.0"
            elif version == "II":
                return "SATA 2.0"
            elif version == "I":
                return "SATA 1.0"
            elif version in ["3", "3.0"]:
                return "SATA 3.0"
            elif version in ["4", "4.0"]:
                return "SATA 4.0"
            elif version in ["5", "5.0"]:
                return "SATA 5.0"
            elif version in ["6", "6.0"]:
                return "SATA 6.0"
            else:
                return f"SATA {version}"
        return "SATA"

    # Standardize PCIe
    pcie_match = re.search(r"\bPCIE\s*([\d.]+)?", io_type)
    if pcie_match:
        version = pcie_match.group(1)
        if version:
            return f"PCIE {version}"
        return "PCIE"

    # Standardize other common I/O types
    elif re.search(r"\b(ETHERNET|LAN|RJ45)\b", io_type):
        return "ETHERNET"
    elif re.search(r"\b(RS-?232|RS-?485|RS-?422|UART|COM|SERIAL)\b", io_type):
        return "SERIAL"
    elif re.search(r"\b(HDMI|VGA|DVI|DISPLAYPORT|DP)\b", io_type):
        return "DISPLAY"
    elif re.search(r"\b(GPIO|DIGITAL I/O)\b", io_type):
        return "GPIO"
    elif re.search(r"\b(I2C|SPI|CAN|SDIO)\b", io_type):
        return "OTHER_BUS"
    elif re.search(r"\b(AUDIO|MIC|LINE-IN|LINE-OUT)\b", io_type):
        return "AUDIO"
    else:
        return "OTHER"


def clean_io_count(value):
    if isinstance(value, str):
        try:
            value_list = ast.literal_eval(value)
            if isinstance(value_list, list):
                if value_list == ["Not Available"]:
                    return []

                # Extract I/O types using regex
                io_types = re.findall(r"\b[\w\s.-]+\b", " ".join(value_list))

                # Standardize and filter I/O types
                return list(set(standardize_io_type(io) for io in io_types if io))
        except:
            # If ast.literal_eval fails, try to extract I/O types directly from the string
            io_types = re.findall(r"\b[\w\s.-]+\b", value)
            return list(set(standardize_io_type(io) for io in io_types if io))
    return value if isinstance(value, list) else []


# Apply the cleaning function
df["io_count"] = df["io_count"].apply(clean_io_count)

# Count unique IO configurations, treating each list as a single entity
unique_io_configs = df["io_count"].apply(lambda x: tuple(sorted(x))).nunique()
print(f"Number of unique IO configurations: {unique_io_configs}")

# Count occurrences of specific IO types
io_type_counts = df["io_count"].explode().value_counts()
print("\nTop 20 most common IO types:")
print(io_type_counts.head(20))

# Count number of IO types per product
io_count_per_product = df["io_count"].apply(len)
print(f"\nAverage number of IO types per product: {io_count_per_product.mean():.2f}")
print(f"Median number of IO types per product: {io_count_per_product.median()}")
print(f"Max number of IO types per product: {io_count_per_product.max()}")

Number of unique IO configurations: 280

Top 20 most common IO types:
io_count
USB          414
OTHER        321
SERIAL       233
ETHERNET     206
USB 3.0      186
USB 2.0      185
DISPLAY      143
PCIE         138
SATA         104
GPIO          81
USB 3.2       65
OTHER_BUS     60
AUDIO         50
USB 3.1       43
SATA 3.0      38
USB 1.0       31
SATA 6.0      13
SATA 2.0      11
USB 4.0        7
PCIE 4         5
Name: count, dtype: int64

Average number of IO types per product: 2.39
Median number of IO types per product: 2.0
Max number of IO types per product: 8


In [17]:
df["io_count"].value_counts()

io_count
[USB]                                                                    114
[USB 3.0]                                                                 77
[USB 2.0]                                                                 72
[]                                                                        68
[USB 1.0]                                                                 28
[USB 3.2]                                                                 27
[OTHER]                                                                   24
[USB 3.1]                                                                 21
[OTHER, USB]                                                              17
[SERIAL, USB]                                                             12
[SATA, ETHERNET, USB, PCIE]                                               12
[SERIAL, USB, OTHER]                                                      11
[GPIO]                                                             

### 2.13 Wireless

This cleaning process will:
- Standardizes Wi-Fi versions (Wi-Fi 6, Wi-Fi 5, and generic Wi-Fi).
- Standardizes Bluetooth versions (Bluetooth 5+, Bluetooth 4, and generic Bluetooth).
- Standardizes cellular technologies (5G, 4G/LTE, 3G, and generic cellular).
- Identifies other common wireless technologies (GPS, NFC, Zigbee, LoRa).
- Groups less common or unspecified wireless technologies as "OTHER".

In [18]:
def standardize_wireless(item):
    item = item.upper().strip()

    # Standardize Wi-Fi
    if re.search(r"\b(WI-?FI|WLAN|802\.11)\b", item):
        if re.search(r"\b(6E?|AX)\b", item):
            return "WI-FI 6"
        elif re.search(r"\b(6|AX)\b", item):
            return "WI-FI 6"
        elif re.search(r"\b(AC|5)\b", item):
            return "WI-FI 5"
        else:
            return "WI-FI"

    # Standardize Bluetooth
    elif re.search(r"\bBLUETOOTH\b", item):
        if re.search(r"\b(5\.\d|LE)\b", item):
            return "BLUETOOTH 5+"
        elif re.search(r"\b4\.\d\b", item):
            return "BLUETOOTH 4"
        else:
            return "BLUETOOTH"

    # Standardize Cellular
    elif re.search(r"\b(5G|LTE|4G|3G|CELLULAR|WWAN)\b", item):
        if "5G" in item:
            return "5G"
        elif "LTE" in item or "4G" in item:
            return "4G/LTE"
        elif "3G" in item:
            return "3G"
        else:
            return "CELLULAR"

    # Other common wireless technologies
    elif "GPS" in item:
        return "GPS"
    elif "NFC" in item:
        return "NFC"
    elif "ZIGBEE" in item:
        return "ZIGBEE"
    elif "LORA" in item:
        return "LORA"

    # For items that don't match any specific category
    else:
        return "OTHER"


def clean_wireless(value):
    if isinstance(value, str):
        try:
            value_list = ast.literal_eval(value)
            if isinstance(value_list, list):
                return list(set(standardize_wireless(item) for item in value_list if item.strip()))
        except:
            return list(set(standardize_wireless(item) for item in value.split(",") if item.strip()))
    return value if isinstance(value, list) else []


# Apply the cleaning function
df["wireless"] = df["wireless"].apply(clean_wireless)

# Display the updated value counts
print(df["wireless"].value_counts().head(20))

# Count unique wireless configurations
unique_wireless_configs = df["wireless"].apply(lambda x: tuple(sorted(x))).nunique()
print(f"\nNumber of unique wireless configurations: {unique_wireless_configs}")

# Count number of wireless types per product
wireless_count_per_product = df["wireless"].apply(len)
print(f"\nAverage number of wireless types per product: {wireless_count_per_product.mean():.2f}")
print(f"Median number of wireless types per product: {wireless_count_per_product.median()}")
print(f"Max number of wireless types per product: {wireless_count_per_product.max()}")

wireless
[]                                320
[WI-FI]                           189
[WI-FI, BLUETOOTH]                111
[OTHER]                            84
[WI-FI 5]                          27
[WI-FI 6]                          26
[WI-FI, CELLULAR, BLUETOOTH]       22
[WI-FI 5, BLUETOOTH]               15
[5G]                               14
[4G/LTE, WI-FI, BLUETOOTH]         14
[WI-FI, OTHER, BLUETOOTH]          12
[BLUETOOTH 5+, WI-FI]               9
[WI-FI 5, BLUETOOTH 4]              8
[WI-FI 5, BLUETOOTH 5+]             7
[WI-FI, BLUETOOTH 4]                7
[WI-FI, 3G]                         6
[WI-FI, OTHER]                      6
[WI-FI 5, OTHER, BLUETOOTH 5+]      6
[4G/LTE]                            6
[4G/LTE, WI-FI]                     5
Name: count, dtype: int64

Number of unique wireless configurations: 79

Average number of wireless types per product: 1.12
Median number of wireless types per product: 1.0
Max number of wireless types per product: 5


In [19]:
df["wireless"].value_counts()

wireless
[]                                        320
[WI-FI]                                   189
[WI-FI, BLUETOOTH]                        111
[OTHER]                                    84
[WI-FI 5]                                  27
[WI-FI 6]                                  26
[WI-FI, CELLULAR, BLUETOOTH]               22
[WI-FI 5, BLUETOOTH]                       15
[5G]                                       14
[4G/LTE, WI-FI, BLUETOOTH]                 14
[WI-FI, OTHER, BLUETOOTH]                  12
[BLUETOOTH 5+, WI-FI]                       9
[WI-FI 5, BLUETOOTH 4]                      8
[WI-FI 5, BLUETOOTH 5+]                     7
[WI-FI, BLUETOOTH 4]                        7
[WI-FI, 3G]                                 6
[WI-FI, OTHER]                              6
[WI-FI 5, OTHER, BLUETOOTH 5+]              6
[4G/LTE]                                    6
[4G/LTE, WI-FI]                             5
[WI-FI, GPS, BLUETOOTH]                     5
[WI-FI 6, BLUETOOTH 5+]  

### 2.14 Operating System BSP

This cleaning process will:
- Standardizes Windows versions (Windows IoT, Windows Embedded, Windows 10, Windows 11, Windows Server, and generic Windows).
- Standardizes Linux distributions (Ubuntu, Yocto Linux, Debian, Fedora, CentOS, Red Hat Enterprise Linux, and generic Linux).
- Identifies other common operating systems (Android, VxWorks, QNX, Raspberry Pi OS).
- Groups less common or unspecified operating systems as "OTHER".

In [20]:
def standardize_os(item):
    item = item.upper().strip()

    # Standardize Windows
    if "WINDOWS" in item:
        if "IOT" in item:
            return "WINDOWS IOT"
        elif "EMBEDDED" in item:
            return "WINDOWS EMBEDDED"
        elif "10" in item:
            return "WINDOWS 10"
        elif "11" in item:
            return "WINDOWS 11"
        elif "SERVER" in item:
            return "WINDOWS SERVER"
        else:
            return "WINDOWS"

    # Standardize Linux
    elif "LINUX" in item:
        if "UBUNTU" in item:
            return "UBUNTU"
        elif "YOCTO" in item:
            return "YOCTO LINUX"
        elif "DEBIAN" in item:
            return "DEBIAN"
        elif "FEDORA" in item:
            return "FEDORA"
        elif "CENTOS" in item:
            return "CENTOS"
        elif "REDHAT" in item or "RED HAT" in item:
            return "RED HAT ENTERPRISE LINUX"
        else:
            return "LINUX"

    # Other common operating systems
    elif "ANDROID" in item:
        return "ANDROID"
    elif "VXWORKS" in item:
        return "VXWORKS"
    elif "QNX" in item:
        return "QNX"
    elif "RASPBIAN" in item or "RASPBERRY PI OS" in item:
        return "RASPBERRY PI OS"

    # For items that don't match any specific category
    else:
        return "OTHER"


def clean_operating_system_bsp(value):
    if isinstance(value, str):
        try:
            value_list = ast.literal_eval(value)
            if isinstance(value_list, list):
                return list(set(standardize_os(item) for item in value_list if item.strip()))
        except:
            return list(set(standardize_os(item) for item in value.split(",") if item.strip()))
    return value if isinstance(value, list) else []


# Apply the cleaning function
df["operating_system_bsp"] = df["operating_system_bsp"].apply(clean_operating_system_bsp)

# Display the updated value counts
print(df["operating_system_bsp"].value_counts().head(20))

# Count unique OS configurations
unique_os_configs = df["operating_system_bsp"].apply(lambda x: tuple(sorted(x))).nunique()
print(f"\nNumber of unique OS configurations: {unique_os_configs}")

# Count number of OS types per product
os_count_per_product = df["operating_system_bsp"].apply(len)
print(f"\nAverage number of OS types per product: {os_count_per_product.mean():.2f}")
print(f"Median number of OS types per product: {os_count_per_product.median()}")
print(f"Max number of OS types per product: {os_count_per_product.max()}")


operating_system_bsp
[]                                                  109
[OTHER]                                              92
[LINUX]                                              67
[LINUX, WINDOWS]                                     61
[ANDROID, LINUX]                                     42
[WINDOWS]                                            28
[OTHER, WINDOWS 10]                                  25
[WINDOWS 10]                                         23
[OTHER, ANDROID, LINUX]                              23
[RASPBERRY PI OS]                                    22
[WINDOWS EMBEDDED]                                   21
[OTHER, LINUX]                                       19
[LINUX, WINDOWS 10]                                  18
[LINUX, WINDOWS IOT]                                 18
[LINUX, WINDOWS EMBEDDED]                            18
[VXWORKS, LINUX, WINDOWS]                            18
[WINDOWS IOT]                                        17
[LINUX, WINDOWS, VXWORKS, Q

In [21]:
df["operating_system_bsp"].value_counts()

operating_system_bsp
[]                                                                    109
[OTHER]                                                                92
[LINUX]                                                                67
[LINUX, WINDOWS]                                                       61
[ANDROID, LINUX]                                                       42
[WINDOWS]                                                              28
[OTHER, WINDOWS 10]                                                    25
[WINDOWS 10]                                                           23
[OTHER, ANDROID, LINUX]                                                23
[RASPBERRY PI OS]                                                      22
[WINDOWS EMBEDDED]                                                     21
[OTHER, LINUX]                                                         19
[LINUX, WINDOWS 10]                                                    18
[LINUX, WINDOWS I

### 2.15 Operating Temperature Max

This cleaning process will:


In [26]:
def clean_temperature(value):
    if isinstance(value, str):
        value = value.upper()
        value = re.sub(r"\s+", "", value)  # Remove all spaces
        # Standardize temperature format
        value = re.sub(r"(\d+(?:\.\d+)?)°?C", r"\1°C", value)
        value = re.sub(r"(\d+(?:\.\d+)?)°?F", r"\1°F", value)
        # Handle 'DEGREESCELSIUS' format
        value = re.sub(r"(\d+(?:\.\d+)?)DEGREESCELSIUS", r"\1°C", value)
        # Handle Fahrenheit to Celsius conversion
        f_to_c_match = re.search(r"(\d+(?:\.\d+)?)°F\((\d+(?:\.\d+)?)°C\)", value)
        if f_to_c_match:
            return f"{f_to_c_match.group(2)}°C"
        # Handle Celsius to Fahrenheit (remove Fahrenheit)
        c_to_f_match = re.search(r"(\d+(?:\.\d+)?)°C\(\d+(?:\.\d+)?°F\)", value)
        if c_to_f_match:
            return f"{c_to_f_match.group(1)}°C"
        # Handle ranges
        range_match = re.search(r"(-?\d+(?:\.\d+)?)°?C\s*(?:TO|-)\s*(-?\d+(?:\.\d+)?)°?C", value)
        if range_match:
            return f"{range_match.group(1)}°C-{range_match.group(2)}°C"
        # Handle single Celsius values
        celsius_match = re.search(r"(-?\d+(?:\.\d+)?)°?C", value)
        if celsius_match:
            return f"{celsius_match.group(1)}°C"
    return value


original_unique = df["operating_temperature_max"].nunique()
df["operating_temperature_max"] = df["operating_temperature_max"].apply(clean_temperature)
cleaned_unique = df["operating_temperature_max"].nunique()

# Compare the number of unique values before and after cleaning
print(f"\nOriginal unique operating temperature max: {original_unique}")
print(f"Cleaned unique operating temperature max: {cleaned_unique}")
print(f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)")

df["operating_temperature_max"].value_counts()


Original unique operating temperature max: 48
Cleaned unique operating temperature max: 21
Reduction: 27 (56.25%)


operating_temperature_max
60°C     504
85°C     340
70°C      54
50°C      18
40°C      15
75°C       6
125°C      4
55°C       4
105°C      3
35°C       3
100°C      2
85℃        2
83°C       1
+80℃       1
65°C       1
90°C       1
45°C       1
80°C       1
85OC       1
95°C       1
70℃        1
Name: count, dtype: int64

### 2.16 Operating Temperature Min


In [27]:
# Apply the cleaning function
original_unique = df["operating_temperature_min"].nunique()
df["operating_temperature_min"] = df["operating_temperature_min"].apply(clean_temperature)
cleaned_unique = df["operating_temperature_min"].nunique()

# Compare the number of unique values before and after cleaning
print(f"\nOriginal unique operating temperature min: {original_unique}")
print(f"Cleaned unique operating temperature min: {cleaned_unique}")
print(
    f"Reduction: {original_unique - cleaned_unique} ({((original_unique - cleaned_unique) / original_unique) * 100:.2f}%)"
)

df["operating_temperature_min"].value_counts()


Original unique operating temperature min: 27
Cleaned unique operating temperature min: 10
Reduction: 17 (62.96%)


operating_temperature_min
-40°C    384
0°C      373
-20°C    161
-25°C     17
-30°C     10
-10°C      8
-40℃       3
5°C        2
-40OC      1
-20℃       1
Name: count, dtype: int64


### 2.17 Certifications

This cleaning process will:
- Standardizes common certifications (CE, FCC, ROHS, UL, CCC, BSMI, CB, MIL-STD-810, EN 50155, IEC 60068, REACH, WEEE, ISO, VCCI, KC, TELEC, IC, RCM).
- Distinguishes between FCC Class A and FCC Class B.
- Groups less common or unspecified certifications as "OTHER".

In [23]:
def standardize_certification(cert):
    cert = cert.upper().strip()

    # Common certifications
    if re.search(r"\bCE\b", cert):
        return "CE"
    elif re.search(r"\bFCC\b", cert):
        if "CLASS A" in cert:
            return "FCC CLASS A"
        elif "CLASS B" in cert:
            return "FCC CLASS B"
        else:
            return "FCC"
    elif re.search(r"\bROHS\b", cert):
        return "ROHS"
    elif re.search(r"\bUL\b", cert):
        return "UL"
    elif re.search(r"\bCCC\b", cert):
        return "CCC"
    elif re.search(r"\bBSMI\b", cert):
        return "BSMI"
    elif re.search(r"\bCB\b", cert):
        return "CB"
    elif re.search(r"\bMIL-STD-810[GH]?\b", cert):
        return "MIL-STD-810"
    elif re.search(r"\bEN\s*50155\b", cert):
        return "EN 50155"
    elif re.search(r"\bIEC\s*60068\b", cert):
        return "IEC 60068"
    elif re.search(r"\bREACH\b", cert):
        return "REACH"
    elif re.search(r"\bWEEE\b", cert):
        return "WEEE"
    elif re.search(r"\bISO\s*\d+", cert):
        return "ISO"
    elif re.search(r"\bVCCI\b", cert):
        return "VCCI"
    elif re.search(r"\bKC\b", cert):
        return "KC"
    elif re.search(r"\bTELEC\b", cert):
        return "TELEC"
    elif re.search(r"\bIC\b", cert):
        return "IC"
    elif re.search(r"\bRCM\b", cert):
        return "RCM"

    # For items that don't match any specific category
    return "OTHER"


def clean_certifications(value):
    if isinstance(value, str):
        try:
            value_list = ast.literal_eval(value)
            if isinstance(value_list, list):
                return list(set(standardize_certification(item) for item in value_list if item.strip()))
        except:
            return list(set(standardize_certification(item) for item in re.split(r"[,;]", value) if item.strip()))
    return value if isinstance(value, list) else []


# Apply the cleaning function
df["certifications"] = df["certifications"].apply(clean_certifications)

# Display the updated value counts
print(df["certifications"].value_counts().head(20))

# Count unique certification configurations
unique_cert_configs = df["certifications"].apply(lambda x: tuple(sorted(x))).nunique()
print(f"\nNumber of unique certification configurations: {unique_cert_configs}")

# Count number of certifications per product
cert_count_per_product = df["certifications"].apply(len)
print(f"\nAverage number of certifications per product: {cert_count_per_product.mean():.2f}")
print(f"Median number of certifications per product: {cert_count_per_product.median()}")
print(f"Max number of certifications per product: {cert_count_per_product.max()}")

certifications
[]                                                221
[CE, FCC]                                          67
[ROHS]                                             42
[ROHS COMPLIANT]                                   31
[CE, FCC CLASS B]                                  29
[CE/FCC]                                           22
[CE, FCC, ROHS]                                    15
[CE, FCC CLASS B, ROHS COMPLIANT]                  13
[CE, FCC CLASS A]                                  13
[ROHS, CE, FCC CLASS B]                            13
[FCC CLASS B]                                      11
[CE/FCC CLASS B]                                   10
[CE, FCC, UL, CCC, BSMI]                           10
[ROHS COMPLIANT 2002/95/EC]                        10
[IEC 60068-2-27, IEC 60068-2-64, MIL-STD-810F]      9
[FCC CLASS A]                                       9
[CE]                                                9
[MIL-STD-810G]                                      8
[FCC, CE]    

In [24]:
df["certifications"].value_counts()

certifications
[]                                                                                                                                                                                                                                                                                                                                                                                                                                                       221
[CE, FCC]                                                                                                                                                                                                                                                                                                                                                                                                                                                 67
[ROHS]                                                                                         

## Cleaned Data

In [30]:
# Display top 10 unique values for each column, sorted by frequency
for column in columns:
    if df[column].dtype == 'object':
        # Handle columns with list values
        if df[column].apply(lambda x: isinstance(x, list)).any():
            # Convert lists to tuples for hashing
            value_counts = df[column].apply(lambda x: tuple(x) if isinstance(x, list) else x).value_counts().head(50)
        else:
            value_counts = df[column].value_counts().head(50)
    else:
        value_counts = df[column].value_counts().head(50)

    unique_count = len(value_counts)

    print(f"\n{'-' * 100}")
    print(f"{column.upper()} (Total unique values: {unique_count})")
    print(f"{'-' * 100}")

    if not value_counts.empty:
        max_value_length = max(len(str(value)) for value in value_counts.index)
        max_count_length = max(len(str(count)) for count in value_counts.values)

        for value, count in value_counts.items():
            percentage = (count / len(df)) * 100
            print(f"{str(value):<{max_value_length}} | {count:>{max_count_length}} | {percentage:.2f}%")
    else:
        print("No data available for this column")

    print(f"{'-' * 100}")

    # Print values as concatenated string
    concatenated_values = ", ".join(str(value) for value in value_counts.index)
    print(f"Values: {concatenated_values}...")
    print()


----------------------------------------------------------------------------------------------------
NAME (Total unique values: 50)
----------------------------------------------------------------------------------------------------
AIMB                                             | 47 | 4.80%
SOM                                              | 20 | 2.04%
ASMB                                             | 12 | 1.23%
PCM                                              | 12 | 1.23%
ARK                                              | 11 | 1.12%
MIO                                              | 10 | 1.02%
RSB                                              |  9 | 0.92%
RASPBERRY-PI MODEL B                             |  7 | 0.72%
VENICE GW                                        |  7 | 0.72%
AIMB-275                                         |  7 | 0.72%
EDHMIC                                           |  6 | 0.61%
COM-EXPRESS COMPACT                              |  6 | 0.61%
COM-EXPRESS BASIC     

In [ ]:
# save cleaned data
df.to_csv("../data/cleaned_data.csv", index=False)